In [1]:
import torch.nn as nn
import torchvision
import torch
import numpy as np
import torchvision.transforms as transforms
from torch.autograd import Variable

import math
import torch.nn.functional as F

In [2]:
class myModel(nn.Module):
    def __init__(self, mode = "hier", dropout = 0.0):
        super().__init__()
        self.drop = dropout
        self.mode = mode
        
        # Convolution 1+2
        if mode == "combined2":
            self.cnn1 = nn.Conv2d(in_channels=4, out_channels=6, kernel_size=5, stride=1, padding=0)
        else:
            self.cnn1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5, stride=1, padding=0)
        self.bn1 = nn.BatchNorm2d(num_features=6)
        self.relu1 = nn.ReLU()
        self.cnn2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1, padding=0)
        self.bn2 = nn.BatchNorm2d(16)
        self.relu2 = nn.ReLU()
        # Max pool 1
        self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2)
     
        # Convolution 3+4
        self.cnn3 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=0)
        self.bn3 = nn.BatchNorm2d(32)
        self.relu3 = nn.ReLU()
        self.cnn4 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, stride=1, padding=0)
        self.bn4 = nn.BatchNorm2d(64)
        self.relu4 = nn.ReLU()
        # Max pool 2
        self.maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        #fc for label 1
        if mode != "cnn":
            self.fcL11 = nn.Linear(19*19*64, 32)
            self.reluL1 = nn.ReLU()
            self.dropout1 = nn.Dropout(p=self.drop)
            self.fcL12 = nn.Linear(32, 6)
        
        # Convolution 5+6
        self.cnn5 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=4, stride=1, padding=0)
        self.bn5 = nn.BatchNorm2d(128)
        self.relu5 = nn.ReLU()
        self.cnn6 = nn.Conv2d(in_channels=128, out_channels=64, kernel_size=3, stride=1, padding=0)
        self.bn6 = nn.BatchNorm2d(64)
        self.relu6 = nn.ReLU()
        
        # fc for label 2
        if mode != "cnn":
            self.fcL21 = nn.Linear(14*14*64, 32)
            self.reluL2 = nn.ReLU()
            self.dropout2 = nn.Dropout(p=self.drop)
            self.fcL22 = nn.Linear(32, 20) 
        
        
        # Convolution 7+8
        self.cnn7 = nn.Conv2d(in_channels=64, out_channels=32, kernel_size=3, stride=1, padding=0)
        self.bn7 = nn.BatchNorm2d(32)
        self.relu7 = nn.ReLU()
        self.cnn8 = nn.Conv2d(in_channels=32, out_channels=16, kernel_size=3, stride=1, padding=0)
        self.bn8 = nn.BatchNorm2d(16)
        self.relu8 = nn.ReLU()

        # fc 1
        self.fc1 = nn.Linear(16*10*10, 120) 
        self.relu9 = nn.ReLU()
        self.dropout3 = nn.Dropout(p=self.drop)

        if mode != "combined1":
            # fc for label 3
            self.fcL3 = nn.Linear(120, 82) 
        else:
            #pose net
            # Fully connected 1
            self.Pfc1 = nn.Linear(17*3, 120) 
            self.Prelu1 = nn.ReLU()
            self.Pdropout1 = nn.Dropout(p=self.drop)

            # Fully connected 2
            self.Pfc2 = nn.Linear(120, 240) 
            self.Prelu2 = nn.ReLU()
            self.Pdropout2 = nn.Dropout(p=self.drop)

            # Fully connected 3
            self.Pfc3 = nn.Linear(240, 120) 
            self.Prelu3 = nn.ReLU()
            self.Pdropout3 = nn.Dropout(p=self.drop)

            #combine
            # Fully connected 4
            self.Cfc1 = nn.Linear(120*2, 120) 
            self.Crelu1 = nn.ReLU()
            self.Cdropout1 = nn.Dropout(p=self.drop)

            # Fully connected 5
            self.Cfc2 = nn.Linear(120, 240) 
            self.Crelu2 = nn.ReLU()
            self.Cdropout2 = nn.Dropout(p=self.drop)

            # Fully connected 6
            self.Cfc3 = nn.Linear(240, 82) 
    
    
    def forward(self, x, y=0.0):
        # Convolution 1+2
        out = self.cnn1(x)
        out = self.bn1(out)
        out = self.relu1(out)
        out = self.cnn2(out)
        out = self.bn2(out)
        out = self.relu2(out)
        out = self.maxpool1(out)
        
        # Convolution 3+4
        out = self.cnn3(out)
        out = self.bn3(out)
        out = self.relu3(out)
        out = self.cnn4(out)
        out = self.bn4(out)
        out = self.relu4(out)
        out = self.maxpool2(out)
        
        # label 1
        if self.mode != "cnn":
            outFlat1 = out.reshape(out.size(0), -1)
            outA = self.fcL11(outFlat1)
            outA = self.reluL1(outA)
            outA = self.dropout1(outA)
            outA = self.fcL12(outA)
        
        # Convolution 5+6
        out = self.cnn5(out)
        out = self.bn5(out)
        out = self.relu5(out)
        out = self.cnn6(out)
        out = self.bn6(out)
        out = self.relu6(out)
        
        
        # label 2
        if self.mode != "cnn":
            outFlat2 = out.reshape(out.size(0), -1)
            outB = self.fcL21(outFlat2)
            outB = self.reluL2(outB)
            outB = self.dropout2(outB)
            outB = self.fcL22(outB)
        
        
        # Convolution 7+8
        out = self.cnn7(out)
        out = self.bn7(out)
        out = self.relu7(out)
        out = self.cnn8(out)
        out = self.bn8(out)
        out = self.relu8(out)

        #Flatten
        out = out.reshape(out.size(0), -1)

        #Dense
        out = self.fc1(out)
        out = self.relu9(out)
        out = self.dropout3(out)
        if self.mode != "combined1":
            outC = self.fcL3(out)
        else:
            #pose
            out2 = y.reshape(y.size(0), -1)
            #Dense
            out2 = self.Pfc1(out2)
            out2 = self.Prelu1(out2)
            out2 = self.Pdropout1(out2)
            out2 = self.Pfc2(out2)
            out2 = self.Prelu2(out2)
            out2 = self.Pdropout2(out2)
            out2 = self.Pfc3(out2)
            out2 = self.Prelu3(out2)
            out2 = self.Pdropout3(out2)
            #combine
            out = torch.cat((out, out2), dim=1)
            out = self.Cfc1(out)
            out = self.Crelu1(out)
            out = self.Cdropout1(out)
            out = self.Cfc2(out)
            out = self.Crelu2(out)
            out = self.Cdropout2(out)
            outC = self.Cfc3(out)
            
        
        if self.mode == "cnn":
            return outC
        return outA,outB,outC

In [3]:
class Net_pose(nn.Module):
    def __init__(self, dropout = 0.0):
        super().__init__()
        self.drop = dropout
      
        # Fully connected 1
        self.fc1 = nn.Linear(17*3, 120) 
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(p=self.drop)
        
        # Fully connected 2
        self.fc2 = nn.Linear(120, 240) 
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(p=self.drop)
        
        # Fully connected3
        self.fc3 = nn.Linear(240, 480) 
        self.relu3 = nn.ReLU()
        self.dropout3 = nn.Dropout(p=self.drop)
        
        # Fully connected 4
        self.fc4 = nn.Linear(480, 82)
        

    def forward(self, x):
        #Flatten
        out = x.reshape(x.size(0), -1)
        #Dense
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.dropout1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.dropout2(out)
        out = self.fc3(out)
        out = self.relu3(out)
        out = self.dropout3(out)
        out = self.fc4(out)
        
        return out

In [4]:
"""
recontructed model from Yoga-82 paper
and added my combine methodes
    code used: https://github.com/andreasveit/densenet-pytorch
"""
class BasicBlock(nn.Module):
    def __init__(self, in_planes, out_planes, dropRate=0.0):
        super(BasicBlock, self).__init__()
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=1,
                               padding=1, bias=False)
        self.droprate = dropRate
    def forward(self, x):
        out = self.conv1(self.relu(self.bn1(x)))
        if self.droprate > 0:
            out = F.dropout(out, p=self.droprate, training=self.training)
        return torch.cat([x, out], 1)

class BottleneckBlock(nn.Module):
    def __init__(self, in_planes, out_planes, dropRate=0.0):
        super(BottleneckBlock, self).__init__()
        inter_planes = out_planes * 4
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(in_planes, inter_planes, kernel_size=1, stride=1,
                               padding=0, bias=False)
        self.bn2 = nn.BatchNorm2d(inter_planes)
        self.conv2 = nn.Conv2d(inter_planes, out_planes, kernel_size=3, stride=1,
                               padding=1, bias=False)
        self.droprate = dropRate
    def forward(self, x):
        out = self.conv1(self.relu(self.bn1(x)))
        if self.droprate > 0:
            out = F.dropout(out, p=self.droprate, inplace=False, training=self.training)
        out = self.conv2(self.relu(self.bn2(out)))
        if self.droprate > 0:
            out = F.dropout(out, p=self.droprate, inplace=False, training=self.training)
        return torch.cat([x, out], 1)

class TransitionBlock(nn.Module):
    def __init__(self, in_planes, out_planes, dropRate=0.0):
        super(TransitionBlock, self).__init__()
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=1,
                               padding=0, bias=False)
        self.droprate = dropRate
    def forward(self, x):
        out = self.conv1(self.relu(self.bn1(x)))
        if self.droprate > 0:
            out = F.dropout(out, p=self.droprate, inplace=False, training=self.training)
        return F.avg_pool2d(out, 2)

class DenseBlock(nn.Module):
    def __init__(self, nb_layers, in_planes, growth_rate, block, dropRate=0.0):
        super(DenseBlock, self).__init__()
        self.layer = self._make_layer(block, in_planes, growth_rate, nb_layers, dropRate)
    def _make_layer(self, block, in_planes, growth_rate, nb_layers, dropRate):
        layers = []
        for i in range(nb_layers):
            layers.append(block(in_planes+i*growth_rate, growth_rate, dropRate))
        return nn.Sequential(*layers)
    def forward(self, x):
        return self.layer(x)

class DenseNet3(nn.Module):
    def __init__(self, num_classes, mode = "hier", growth_rate=12,
                 reduction=0.5, bottleneck=True, dropRate=0.0, dropoutPose = 0.0):
        super(DenseNet3, self).__init__()
        self.mode = mode
        self.drop = dropoutPose
        in_planes = 2 * growth_rate
        
        if bottleneck == True:
            block = BottleneckBlock
        else:
            block = BasicBlock
        n = [5,5,5,5]#  [6,12,48,32]is used in the Yoga-82 paper
        
        
        
        # 1st conv before any dense block
        if self.mode == "combined2":
            self.conv1 = nn.Conv2d(4, in_planes, kernel_size=3, stride=1,
                               padding=1, bias=False)
        else:
            self.conv1 = nn.Conv2d(3, in_planes, kernel_size=3, stride=1,
                               padding=1, bias=False)
        # 1st block
        self.block1 = DenseBlock(n[0], in_planes, growth_rate, block, dropRate)
        in_planes = int(in_planes+n[0]*growth_rate)
        self.trans1 = TransitionBlock(in_planes, int(math.floor(in_planes*reduction)), dropRate=dropRate)
        in_planes = int(math.floor(in_planes*reduction))
        # 2nd block
        self.block2 = DenseBlock(n[1], in_planes, growth_rate, block, dropRate)
        in_planes = int(in_planes+n[1]*growth_rate)
        self.trans2 = TransitionBlock(in_planes, int(math.floor(in_planes*reduction)), dropRate=dropRate)
        in_planes = int(math.floor(in_planes*reduction))
        
        if self.mode != "cnn":
            self.temp_in_planes1 = in_planes
            self.fcL1 = nn.Linear(625*in_planes, 6)
            
            
        # 3nd block
        self.block3 = DenseBlock(n[2], in_planes, growth_rate, block, dropRate)
        in_planes = int(in_planes+n[2]*growth_rate)
        self.trans3 = TransitionBlock(in_planes, int(math.floor(in_planes*reduction)), dropRate=dropRate)
        in_planes = int(math.floor(in_planes*reduction))
        
        if self.mode != "cnn":
            self.temp_in_planes2 = in_planes
            self.fcL2 = nn.Linear(144*in_planes, 20)
            
            
        # 4rd block
        self.block4 = DenseBlock(n[3], in_planes, growth_rate, block, dropRate)
        in_planes = int(in_planes+n[3]*growth_rate)
        # global average pooling and classifier
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.relu = nn.ReLU(inplace=True)
        if self.mode == "combined1":
            self.fc = nn.Linear(in_planes+84, num_classes)
        else:
            self.fc = nn.Linear(in_planes, num_classes)
        self.in_planes = in_planes


        if self.mode == "combined1":
            #netPose
            # Fully connected 1
            self.Pfc1 = nn.Linear(17*3, 120) 
            self.Prelu1 = nn.ReLU()
            self.Pdropout1 = nn.Dropout(p=self.drop)

            # Fully connected 1
            self.Pfc2 = nn.Linear(120, 240) 
            self.Prelu2 = nn.ReLU()
            self.Pdropout2 = nn.Dropout(p=self.drop)

            # Fully connected 1
            self.Pfc3 = nn.Linear(240, 84) 
            self.Prelu3 = nn.ReLU()
            self.Pdropout3 = nn.Dropout(p=self.drop)


            #combine
            # Fully connected 1
            self.Cfc1 = nn.Linear(84*2, 120) 
            self.Crelu1 = nn.ReLU()
            self.Cdropout1 = nn.Dropout(p=self.drop)

            # Fully connected 1
            self.Cfc2 = nn.Linear(120, 240) 
            self.Crelu2 = nn.ReLU()
            self.Cdropout2 = nn.Dropout(p=self.drop)

            # Fully connected 1
            self.Cfc3 = nn.Linear(240, 20) 
            # Fully connected 1
            self.Cfc4 = nn.Linear(240, 82) 
        
        
            
            
            
            
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.bias.data.zero_()
    def forward(self, x, y = 0):
        out = self.conv1(x)
        out = self.trans1(self.block1(out))
        out = self.trans2(self.block2(out))
        if self.mode != "cnn":
            outFlat = out.reshape(-1, 625*self.temp_in_planes1)
            outL1 = self.fcL1(outFlat)
        out = self.trans3(self.block3(out))
        if self.mode != "cnn":
            outFlat = out.reshape(-1, 144*self.temp_in_planes2)
            outL2 = self.fcL2(outFlat)
        out = self.block4(out)
        out = self.relu(self.bn1(out))
        out = F.avg_pool2d(out, 8)
        out = out.reshape(-1, self.in_planes)
        
        
        if self.mode == "combined1":
            #pose
            out2 = y.reshape(y.size(0), -1)
            out2 = self.Pfc1(out2)
            out2 = self.Prelu1(out2)
            out2 = self.Pdropout1(out2)
            out2 = self.Pfc2(out2)
            out2 = self.Prelu2(out2)
            out2 = self.Pdropout2(out2)
            out2 = self.Pfc3(out2)
            out2 = self.Prelu3(out2)
            out2 = self.Pdropout3(out2)
            #combine
            out = torch.cat((out, out2), dim=1)
            out = self.fc(out)
        else:
            out = self.fc(out)
        if self.mode != "cnn":
            return outL1,outL2,out
        return out